In [16]:
# ! pip show crewai
# ! pip install 'crewai[tools]'
# ! pip install crewai

In [1]:
import os
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.llms import CTransformers
from crewai_tools import BaseTool
from textwrap import dedent
from crewai_tools import DirectoryReadTool, FileReadTool
from datetime import datetime


2024-06-26 06:48:20,797 [embedchain] [INFO] Swapped std-lib sqlite3 with pysqlite3 for ChromaDb compatibility. Your original version was 3.31.1.


In [2]:
import os
import json
from langchain.tools import tool
from pydantic.v1 import BaseModel, validator, Field
from typing import List
import random
import matplotlib.pyplot as plt

In [2]:
# MODEL_PATH = '../mistral-7b-instruct-v0.2.Q4_K_M.gguf'

# config = {
#     "max_new_tokens": 2048,
#     "context_length": 4096,
#     "repetition_penalty": 1.1,
#     "temperature": 0.5,
#     "top_k": 50,
#     "top_p": 0.9,
#     "stream": True,
#     "threads": int(os.cpu_count() / 2)
# }

# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# llm = CTransformers(model=MODEL_PATH, config=config, callback_manager=callback_manager)

In [3]:
os.environ["OPENAI_API_BASE"] = 'http://localhost:5000/v1'
os.environ["OPENAI_MODEL_NAME"] = 'mistral-instruct-7b'
os.environ["OPENAI_API_KEY"] = 'NA'

In [4]:
from langchain.tools import tool

class FolderReadTool():
    @tool("ReadFolder")
    def ReadFolder(folderPath:str) -> list:       
        """
        Reads a folder and returns the list of files        
        """
        try:
            # Check if the folder exists
            if not os.path.isdir(folderPath):
                return f"Error: The folder '{folderPath}' does not exist."

            # Get the list of files in the folder
            files = os.listdir(folderPath)
            # return "\n".join(files) if files else "The folder is empty."
            return files if files else []
        except Exception as e:
            return f"An error occurred: {str(e)}"

In [6]:
directoryPath = '/home/fredsena/FredCodes/GIT/EMAILS/'
folder = FolderReadTool()
result =  folder.ReadFolder(directoryPath)
result

['YouAppearedIn4Searches.txt', 'DoYouHaveStumblingBlocksPitfalls.txt']

In [7]:
# file_reader_tool.py

from langchain.tools import tool

class FileReaderTool():
    @tool("FileReaderTool")
    def ReadFile(filePath:str):
        """
        Reads the contents of the specified file and returns it as a string.
        
        Args:
        filePath (str): The path to the file to be read.
        
        Returns:
        str: The contents of the file.
        """
        try:
            with open(filePath, 'r') as file:
                contents = file.read()
            return contents
        except Exception as e:
            return f"An error occurred while reading the file: {str(e)}"
            
#ReadFileContentTool = FileReaderTool()

In [17]:
file = '/home/fredsena/FredCodes/GIT/EMAILS/DoYouHaveStumblingBlocksPitfalls.txt'
fileReaderTool = FileReaderTool()
fileReaderTool.ReadFile(file)

"Sender: Evan Bachtold <evanbachtold@gmail.com>\nRecipient: undisclosed-recipients:;\nSubject: Do you have any stumbling blocks or pitfalls?\nDate: Sat, 18 May 2024 11:28:07 +0100\n\nIf so, you are like me and look forward to our Men's zoom tomorrow\nmorning.  Only 2 left until summer break.  We will look at Romans 14.  Hope\nyou are able to join 8:00 AM!\n\nhttps://us02web.zoom.us/j/6054416511?omn=81732385349\n\n☕📖🙏\n\nEvan\n"

In [8]:
from pydantic import BaseModel
from typing import List, Dict

class Link(BaseModel):
    url: str
    description: str

class ResultDTO(BaseModel):
    senderEmail: str
    summary: str
    mainPoints: List[str]
    links: List[Link]


In [9]:
class EmailAnalystAgents:
    #def __init__(self):
        # self.OpenAIGPT35 = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.4, api_key=oai_api_key)
        # self.OpenAIGPT4 = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.5)
        # self.Ollama = Ollama(model="openhermes")
        #self.gemini = ChatGoogleGenerativeAI(model="gemini-pro",
        #                       verbose=True, temperature=0.5, google_api_key=gemini_api_key)

    def EmailFilterAgent(self):
        return Agent(
            role='Senior Email Analyst',
            goal=dedent(f"""Filter out non-essential emails like newsletters and promotional content."""),
            backstory=dedent("""\
                As a Senior Email Analyst, you have extensive experience in email content analysis.
                You are adept at distinguishing important emails from spam, newsletters, and other
                irrelevant content. Your expertise lies in identifying key patterns and markers that
                signify the importance of an email."""),
            verbose=True,
            allow_delegation=False,    
            tools=[FolderReadTool.ReadFolder, FileReaderTool.ReadFile],
            #llm=self.OpenAIGPT4,
        )

In [10]:
from crewai import Task
from textwrap import dedent
from typing import List
from pydantic import BaseModel

class EmailActionTasks:    

    def ListFiles(self, agent, data: str):
        return Task(
            description= ("List all files from the folder: {folderPath}. "
                           "Make sure to use everything you know "
                           "to provide the best support possible. "
                    	   "You must strive to provide a complete "
                           "and accurate response to the request."
            ),
            agent=agent,
            #tools=[FolderReadTool.ReadFolder],
            expected_output="""**ONLY** A list of files in a folder. 
                Make sure to provide an accurate list and make no assumptions.""",
            verbose=True,
            #llm=llm
        )

    def AnalyzeEmails(self, agent, context):
        return Task(
            description=dedent("""\
                For each email from the folder: {folderPath}, pull and analyze it.
                Consider any txt file as an email. 
                If no links are found in an email, proceed to the next email. 
                If the email file is a None value, ignore it and proceed to the next one.
                understand the context, key points of the conversation. 
                then start reading the email file, read its content, and then proceed to the next one and so on.
                IMPORTANT: whenever you are going to use FileReaderTool tool, ALWAYS use this path: {folderPath} to read the file.
                """),
            agent=agent,
            context=context,
            expected_output=("""\    
                Your final answer MUST be a json list with:        
                - html links, if provided
                - a short summary of each email
                - a highlighting with the main points
                - identify the user email and who he will be answering to        
                - the sender's email address"""),
                #tools=[FileReaderTool.ReadFile],
                output_file=f"{'result_dto'}_{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.txt",                
                verbose=True,
                #llm=llm
        )

In [11]:
from crewai import Agent, Task, Crew, Process
from textwrap import dedent

class EmailAnalyzerCrew:
    def __init__(self, data):
        self.data = data

    def run(self):
        agents = EmailAnalystAgents()
        tasks = EmailActionTasks()

        # AGENTS
        filter_agent = agents.EmailFilterAgent()

        # TASKS
        list_files_task = tasks.ListFiles(filter_agent, self.data)
        analyze_task = tasks.AnalyzeEmails(filter_agent, [list_files_task])

        # Define your custom crew here
        crew = Crew(
            agents=[filter_agent],
            tasks=[
                list_files_task,
                analyze_task,
                ],
            verbose=True,
            #memory=False,
        )
        inputs = {'folderPath': '/home/fredsena/FredCodes/GIT/EMAILS/'}        
        result = crew.kickoff(inputs=inputs)
        return result

In [13]:
mycrew = EmailAnalyzerCrew("/home/fredsena/FredCodes/GIT/EMAILS/")

In [15]:
result = mycrew.run()

2024-06-26 06:54:23,285 - 140340239537984 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Email Analyst
 [INFO]: == Starting Task: List all files from the folder: /home/fredsena/FredCodes/GIT/EMAILS/. Make sure to use everything you know to provide the best support possible. You must strive to provide a complete and accurate response to the request.


> Entering new CrewAgentExecutor chain...
I need to read the folder and get a list of files in it.
Action: ReadFolder
Action Input: {"folderPath": "/home/fredsena/FredCodes/GIT/EMAILS/"} 

['YouAppearedIn4Searches.txt', 'DoYouHaveStumblingBlocksPitfalls.txt']

Final Answer: ['YouAppearedIn4Searches.txt', 'DoYouHaveStumblingBlocksPitfalls.txt']

> Finished chain.
 [DEBUG]: == [Senior Email Analyst] Task output: ['YouAppearedIn4Searches.txt', 'DoYouHaveStumblingBlocksPitfalls.txt']


 [DEBUG]: == Working Agent: Senior Email Analyst
 [INFO]: == Starting Task: For each email from the folder: /home/fredsena/FredCodes/GIT/EMAILS/, pull and analyze it.
Consider any txt file as an email. 
If no links

In [ ]:
print(result)